<a href="https://colab.research.google.com/github/Sarvesh1814/HateXplain/blob/Sarvesh/CNN_GRU_V7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, GRU, Dense, Dropout
import numpy as np

from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from gensim.models import Word2Vec
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
import tensorflow as tf


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gensim
import spacy
import nltk
from tensorflow.keras.utils import pad_sequences
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, Conv1D, MaxPooling1D, GRU, Dense, Dropout
from keras.models import Sequential
from keras.optimizers import RMSprop,Adam
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer

In [4]:
# Download the pre-trained GloVe word embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2023-04-13 10:19:53--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-04-13 10:19:53--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-04-13 10:19:53--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [5]:
df = pd.read_csv("/content/drive/MyDrive/Reproducibility/Sample Model/HateXplain2.csv")

In [6]:
data = df[['post_tokens',	'Target_cat',	'Label_cat',	'final_rationales']]
data['post_tokens'] = data['post_tokens'].apply(lambda x: eval(x))
for i in range(len(data)):
  
  sentence =""
  for j in (data['post_tokens'].iloc[i]):
    
    sentence += j +" "
    
  data['post_tokens'].iloc[i] = sentence

<ipython-input-6-5f0ee8932d8a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['post_tokens'] = data['post_tokens'].apply(lambda x: eval(x))
<ipython-input-6-5f0ee8932d8a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['post_tokens'].iloc[i] = sentence


In [7]:
label_map = {'hatespeech': 0, 'normal': 1, 'offensive': 2}
labels = data["Label_cat"].apply(lambda x: label_map[x])

In [8]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y2 = label_encoder.fit_transform(data['Target_cat'])

In [9]:
max_words = 30000
maxlen = 100

# Tokenize the text data and convert it to sequences of integers
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['post_tokens'])
sequences = tokenizer.texts_to_sequences(df['post_tokens'])
X = pad_sequences(sequences, maxlen=maxlen)

# Define the target labels
y1 = pd.get_dummies(df['Label_cat']).values
y2 = to_categorical(y2, num_classes=21)

In [10]:
y1

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1]], dtype=uint8)

In [11]:
# Load the pre-trained GloVe embeddings
embedding_dict = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embedding_dict[word] = vector

# Initialize the embedding matrix with pre-trained GloVe embeddings
word_index = tokenizer.word_index
num_words = min(max_words, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, 100))
for word, i in word_index.items():
    if i >= max_words:
        continue
    embedding_vector = embedding_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


## Model Arichtecture

In [14]:
# Define the model architecture
from keras.models import Model
model1 = Sequential()
model1.add(Embedding(num_words, 100, weights=[embedding_matrix], input_length=maxlen, trainable=True))
model1.add(Conv1D(64, 7, activation='relu'))
model1.add(MaxPooling1D(2))
model1.add(Dropout(0.1))
model1.add(Conv1D(128, 7, activation='relu'))
model1.add(MaxPooling1D(2))
model1.add(GRU(128, dropout=0.2, recurrent_dropout=0.1))
model1.add(Dense(128, activation='relu'))
model1.add(Dense(64, activation='relu'))
model2 = Dense(3, activation='softmax')(model1.output)
model3 = Dense(21, activation='softmax')(model1.output)

model = Model(inputs=model1.input, outputs=[model2, model3])
# Compile the model1
model.compile(optimizer=RMSprop(lr=1e-2), loss='categorical_crossentropy', metrics=['acc'])

# Train the model1
history=model.fit(X, [y1, y2], epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
252/252 [==============================] - 54s 198ms/step - loss: 3.0492 - dense_10_loss: 1.0373 - dense_11_loss: 2.0119 - dense_10_acc: 0.4788 - dense_11_acc: 0.4141 - val_loss: 3.3039 - val_dense_10_loss: 1.0447 - val_dense_11_loss: 2.2592 - val_dense_10_acc: 0.3459 - val_dense_11_acc: 0.2400
Epoch 2/10
252/252 [==============================] - 51s 203ms/step - loss: 2.6801 - dense_10_loss: 0.8916 - dense_11_loss: 1.7885 - dense_10_acc: 0.5825 - dense_11_acc: 0.4734 - val_loss: 3.3233 - val_dense_10_loss: 1.1025 - val_dense_11_loss: 2.2209 - val_dense_10_acc: 0.3730 - val_dense_11_acc: 0.2734
Epoch 3/10
252/252 [==============================] - 49s 193ms/step - loss: 2.5486 - dense_10_loss: 0.8532 - dense_11_loss: 1.6954 - dense_10_acc: 0.6075 - dense_11_acc: 0.4988 - val_loss: 3.2736 - val_dense_10_loss: 1.0989 - val_dense_11_loss: 2.1747 - val_dense_10_acc: 0.3806 - val_dense_11_acc: 0.2747
Epoch 4/10
252/252 [==============================] - 51s 204ms/step - loss: 2.

In [15]:
# Get the predicted labels
y_pred1, y_pred2 = model.predict(X)

# Convert the labels from one-hot encoding to integers
y_pred1 = np.argmax(y_pred1, axis=1)
y_pred2 = np.argmax(y_pred2, axis=1)
y_test_int1 = np.argmax(y1, axis=1)
y_test_int2 = np.argmax(y2, axis=1)


630/630 [==============================] - 14s 22ms/step


In [20]:
from sklearn.metrics import classification_report
print("Classification report for model2:")
print(classification_report(y_test_int2,y_pred2))

Classification report for model2:
              precision    recall  f1-score   support

           0       0.77      0.56      0.65      3037
           1       0.00      0.00      0.00       583
           2       0.00      0.00      0.00       326
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00       334
           5       0.00      0.00      0.00        33
           6       0.00      0.00      0.00        48
           7       0.00      0.00      0.00        10
           8       0.00      0.00      0.00        15
           9       0.00      0.00      0.00       222
          10       0.22      0.06      0.09      1576
          11       0.00      0.00      0.00         8
          12       0.00      0.00      0.00         4
          13       0.47      0.40      0.43      1633
          14       0.50      0.64      0.56      1424
          15       0.00      0.00      0.00        89
          16       0.00      0.00      0.00    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
print("Classification report for model3:")
print(classification_report(y_test_int1,y_pred1))

Classification report for model3:
              precision    recall  f1-score   support

           0       0.70      0.67      0.68      5934
           1       0.63      0.79      0.70      7814
           2       0.51      0.38      0.43      6400

    accuracy                           0.62     20148
   macro avg       0.61      0.61      0.61     20148
weighted avg       0.61      0.62      0.61     20148

